# Classification and detection model - Version 1

The classification backbone of the model is VGG-16 network along with the detection activation layers

In [ ]:
import keras
import tensorflow as tf
import keras.backend as K 

## CASTING DATA




### Building model

In [ ]:
def build_model(optimizer, learning_rate, out_type):

    policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')#sets values to be float16 for nvidia 2000,3000 series GPUs, plus others im sure
    
    input_img = keras.layers.Input(shape=(224, 224, 3))
  
    x = keras.layers.Conv2D(8, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(input_img)
    x = keras.layers.Conv2D(8, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Conv2D(16, 2,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Conv2D(32, 2,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(32, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)    

    x = keras.layers.Conv2D(16, 1, activation="selu",kernel_initializer="lecun_normal",dtype=policy)(x)
    x = keras.layers.Conv2D(1, 1, activation='linear')(x)
    x_map = keras.activations.sigmoid(x)

    y = keras.layers.GlobalMaxPooling2D()(x_map)
    
    if optimizer == "SGD":
        #looks like pretty good but noisy results with no momentum, lets check 0.9...
        optimizer = keras.optimizers.SGD(lr=learning_rate)
    if optimizer == "RMSprop":
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    if optimizer == "Adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    if out_type == "classify":
        model = keras.models.Model(inputs=input_img, outputs=y)
    if out_type == "map":
        model = keras.models.Model(inputs=input_img, outputs=x_map)
    model.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['accuracy','AUC'])

    return model


In [ ]:
model = build_model("Adam", 1e-3, "classify")
model.summary()

### Pre-processing


In [ ]:
import os
import tarfile
import shutil
import numpy as np

dataset_dir = '/content/drive/MyDrive/Major_Project/Datasets/datasets'

model_dir = '/content/drive/MyDrive/Major_Project/Models/V1'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [ ]:
filename = 'casting_dataset.tar.xz'
tar_file = os.path.join(dataset_dir, filename)

my_tar = tarfile.open(tar_file)
my_tar.extractall(dataset_dir) # specify which folder to extract to
my_tar.close()

In [ ]:
dataset = os.path.join(dataset_dir, 'casting_dataset_balanced')

In [ ]:
generator = keras.preprocessing.image.ImageDataGenerator(
                            rotation_range=10,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            brightness_range = [0.8,1.0],
                            zoom_range=0.1,
                            rescale=1./255,
                            fill_mode="constant",
                            cval=1.0,
                            horizontal_flip=True,
                            vertical_flip=True,
                            validation_split=0.1,
                            dtype='float64',
                        )


In [ ]:
print(len(os.listdir(os.path.join(dataset, 'train', 'def_front'))))
print(len(os.listdir(os.path.join(dataset, 'train', 'ok_front'))))
print(len(os.listdir(os.path.join(dataset, 'test', 'def_front'))))
print(len(os.listdir(os.path.join(dataset, 'test', 'ok_front'))))

In [ ]:
batch_size = 64

train_generator = generator.flow_from_directory( os.path.join(dataset, 'train'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='binary',
                                                subset='training',
                                                shuffle=True) 
                                            
val_generator = generator.flow_from_directory( os.path.join(dataset, 'train'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='binary',
                                                subset='validation',
                                                shuffle=True) 


### Training model

In [ ]:
epochs = 50

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=5, min_lr = 1e-7, factor=0.5, verbose=1)

history = model.fit(train_generator, 
                    epochs= epochs,
                    validation_data = val_generator,
                    verbose=1,
                    callbacks=[reduce_lr]) 

In [ ]:
model.save(os.path.join(model_dir, 'casting_V4(50eps).h5'))

In [ ]:
import matplotlib.pyplot as plt


plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()


### Testing the model

In [ ]:
model = keras.models.load_model(os.path.join(model_dir, 'casting_V2.h5'))
model.summary()

In [ ]:
batch_size = 64

test_generator = generator.flow_from_directory( os.path.join(dataset, 'test'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='binary') 

In [ ]:
attribute_to_idx = test_generator.class_indices
idx_to_attribute = {value:key for key,value in attribute_to_idx.items()}

print(attribute_to_idx)
print(idx_to_attribute)

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
scores_keys = ['loss', 'accuracy', 'auc']

for key,score in zip(scores_keys, scores):

    print(key, ':', score)

"""
50 eps

loss : 0.11778640002012253
accuracy : 0.9632152318954468
auc : 0.9908440709114075
"""

"""
100 eps

loss : 0.04719378054141998
accuracy : 0.9863760471343994
auc : 0.9984898567199707
"""

#### Visualise test results

In [ ]:
from google.colab.patches import cv2_imshow

for idx, (images, output) in enumerate(test_generator):

    if idx == 2:
        break

    for i in range(batch_size):

        image = images[i]

        cv2_imshow(image * 255)

        preds = model.predict(np.expand_dims(image, axis=0))[0][0]

        actual_value = output[i]

        predicted_value = (preds > 0.75).astype(np.int)

        if predicted_value == 0:

            confidence = (1- preds) * 100
        else:
            confidence = preds * 100

        print("Confidence", confidence, "%")
        print("Actual_value", idx_to_attribute[int(actual_value)])
        print("Predicted value", idx_to_attribute[int(predicted_value)])
    
    idx += 1

#### Get Activation maps

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(
    img_array, model, last_conv_layer, classifier_layer
):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    x = classifier_layer(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.cm as cm

last_conv_layer = model.layers[-3]
classifier_layer = model.layers[-1]

for idx, (images, output) in enumerate(test_generator):

    if idx == 1:
        break

    for i in range(batch_size):
        img = images[i] * 255

        image = np.expand_dims(images[i], axis=0)

        preds = model.predict(image, verbose=1)[0][0]

        actual_value = output[i]

        predicted_value = (preds > 0.6).astype(np.int)

        if predicted_value == 0:

            confidence = (1- preds) * 100
        else:
            confidence = preds * 100

       
        # Generate class activation heatmap
        heatmap = make_gradcam_heatmap(
            image, model, last_conv_layer, classifier_layer
        )


        # We rescale heatmap to a range 0-255
        heatmap = np.uint8(255 * heatmap)

        # We use jet colormap to colorize heatmap
        jet = cm.get_cmap("jet")

        # We use RGB values of the colormap
        jet_colors = jet(np.arange(256))[:, :3]
        jet_heatmap = jet_colors[heatmap]

        # We create an image with RGB colorized heatmap
        jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
        jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
        jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

        # Superimpose the heatmap on original image
        superimposed_img = jet_heatmap * 0.6 + img
        superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

        plt.figure(figsize=(5,5))
        plt.imshow(superimposed_img)
        plt.show()

        print("Confidence", confidence, "%")
        print("Actual_value", idx_to_attribute[int(actual_value)])
        print("Predicted value", idx_to_attribute[int(predicted_value)])
    
        
    idx += 1

#### Visualise detection results

In [ ]:
batch_size = 1

generator = keras.preprocessing.image.ImageDataGenerator(
                            rescale=1./255,
                            dtype='float64',
                        )

test_generator = generator.flow_from_directory( os.path.join(dataset, 'test'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='binary') 
disp_images = []

pos_idx = 0
neg_idx = 0

for image, actual in test_generator:

    output = actual

    if output == 0 and pos_idx < 3:
        pos_idx += 1
        disp_images.append(image[0, ...])

    
    if output == 1 and neg_idx <3:
        neg_idx += 1
        disp_images.append(image[0, ...])

    if neg_idx>=3 and pos_idx>=3:
        break

disp_images = np.array(disp_images)
print(disp_images.shape)

In [ ]:
import matplotlib as mplot 

def plot_maps(images, maps, k, dims):
    map_size=dims[0]
    r_size = dims[1]
    r_stride = dims[2]
    colors = ['#ff0000', '#ff0080', '#ff00ff', '#8000ff', '#0080ff', '#00ffff', '#00ff80']
              
    fig, ax = plt.subplots(2, 3, figsize=(20,15))
    for j , (img, mask) in enumerate(zip(images, maps)):
        mask = mask.flatten()
        j_a = (j-j%3)//3
        j_b = j%3
        img = np.asarray(img[:,:,0],dtype=np.float32)
        ax[j_a][j_b].imshow(img,cmap=plt.get_cmap('gray'))
        
        for i in range(k):
            a_max = np.argmax(mask)
            x_region = a_max%map_size#note, numpy addresses work on arr[y, x, z], compared to image coordinates
            y_region = (a_max-(a_max%map_size))//map_size
            prob = mask[a_max]
       
            if prob>0.5:break
            x_pixel = r_stride*(x_region)
            y_pixel = r_stride*(y_region)
       
            #rectangle expects bottom left coordinates. We've generated Top Right
            rectangle = mplot.patches.Rectangle((x_pixel, y_pixel), r_size, r_size, edgecolor=colors[i-1],facecolor="none")
            ax[j_a][j_b].add_patch(rectangle)
            
            font = {'color':colors[i-1]}
            ax[j_a][j_b].text(x_pixel,y_pixel,s="{0:.3f}".format(prob), fontdict=font)
            
            mask[a_max]= 0#to help get the next most maximum
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()

In [ ]:
map_model = keras.models.Model(inputs=[model.input], 
                               outputs=[model.layers[-2].output])

final_layer = model.layers[-1]

pred_maps = map_model.predict(disp_images)

plot_maps(disp_images, pred_maps, 5,  (15, 32, 14 ))

## SOLAR PANEL DATA 


### Building model

In [ ]:
import keras
import tensorflow as tf
import keras.backend as K 

In [ ]:
def build_model(optimizer, learning_rate, out_type, categories):

    policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')#sets values to be float16 for nvidia 2000,3000 series GPUs, plus others im sure
    
    input_img = keras.layers.Input(shape=(224, 224, 3))
  
    x = keras.layers.Conv2D(8, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(input_img)
    x = keras.layers.Conv2D(8, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Conv2D(16, 2,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Conv2D(32, 2,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(32, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)    

    x = keras.layers.Conv2D(16, 1, activation="selu",kernel_initializer="lecun_normal",dtype=policy)(x)
    x = keras.layers.Conv2D(1, 1, activation='linear')(x)
    x_map = keras.activations.sigmoid(x)

    y = keras.layers.Flatten()(x_map)
    y_out = keras.layers.Dense(categories, activation='softmax')(y)
    
    if optimizer == "SGD":
        #looks like pretty good but noisy results with no momentum, lets check 0.9...
        optimizer = keras.optimizers.SGD(lr=learning_rate)
    if optimizer == "RMSprop":
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    if optimizer == "Adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    if out_type == "classify":casting_V2
        model = keras.models.Model(inputs=input_img, outputs=y_out)
    if out_type == "map":
        model = keras.models.Model(inputs=input_img, outputs=x_map)
    model.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['accuracy','AUC'])

    return model


In [ ]:
map_model = build_model("Adam", 1e-3, "map", 2)
model = build_model("Adam", 1e-3, "classify", 2)
model.summary()

### Pre-processing

In [ ]:
import os
import tarfile
import shutil
import numpy as np

dataset_dir = '/content/drive/MyDrive/Major Project/Datasets/datasets'

model_dir = '/content/drive/MyDrive/Major Project/Models/V1'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [ ]:
filename = 'solar_panels_products.tar.xz'
tar_file = os.path.join(dataset_dir, filename)

my_tar = tarfile.open(tar_file)
my_tar.extractall(dataset_dir) # specify which folder to extract to
my_tar.close()

In [ ]:
dataset = os.path.join(dataset_dir, 'solar_panels_products')

In [ ]:
generator = keras.preprocessing.image.ImageDataGenerator(
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            brightness_range = [0.5,1.0],
                            zoom_range=0.1,
                            rescale=1./255,
                            fill_mode="nearest",
                            cval=0.0,
                            horizontal_flip=True,
                            vertical_flip=True,
                            validation_split=0.75,
                            dtype='float64',
                        )

In [ ]:
batch_size = 64

train_generator = generator.flow_from_directory( os.path.join(dataset, 'train'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='training',
                                                shuffle=True) 
                                            
val_generator = generator.flow_from_directory( os.path.join(dataset, 'train'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='validation',
                                                shuffle=True) 

### Training the model

In [ ]:
epochs = 50

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=3, min_lr = 1e-7, factor=0.5, verbose=1)

history = model.fit(train_generator, 
                    epochs= epochs,
                    validation_data = val_generator,
                    verbose=1,
                    callbacks=[reduce_lr])

In [ ]:
import matplotlib.pyplot as plt

print("AUC")
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.show()

print("Loss")
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

print('Accuracy')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()


print('Learning rate')
plt.plot(history.history['lr'])
plt.show()

In [ ]:
model.save(os.path.join(model_dir, 'solar_v1(50eps).h5'))

### Testing the model

In [ ]:
import keras
import os
model_dir = '/content/drive/MyDrive/Major Project/Models/V1'

model = keras.models.load_model(os.path.join(model_dir, 'solar_v1(50eps).h5'))
model.summary()

In [ ]:
test_generator = generator.flow_from_directory( os.path.join(dataset, 'test'), 
                                                target_size=(224, 224), 
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=True) 

In [ ]:
attribute_to_idx = test_generator.class_indices
idx_to_attribute = {value:key for key,value in attribute_to_idx.items()}

print(attribute_to_idx)
print(idx_to_attribute)

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
scores_keys = ['loss', 'accuracy', 'auc']

for key,score in zip(scores_keys, scores):

    print(key, ':', score)

"""
50 epochs

loss : 0.38369879126548767
accuracy : 0.9211195707321167
auc : 0.9531559944152832
"""

#### Visualise test data results

In [ ]:
from google.colab.patches import cv2_imshow

for idx, (images, output) in enumerate(test_generator):

    if idx == 20:
        break

    for i in range(1):

        image = images[i]

        cv2_imshow(image * 255)

        preds = model.predict(np.expand_dims(image, axis=0))

        actual_value = np.argmax(output[i], axis=0)

        predicted_value = np.argmax(preds, axis=1)[0]

        confidence = preds[0][predicted_value] * 100

        print("Confidence", confidence, "%")
        print("Actual_value", idx_to_attribute[int(actual_value)])
        print("Predicted value", idx_to_attribute[int(predicted_value)])
    
    idx += 1

#### Get Activation Maps

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(
    img_array, model, last_conv_layer, classifier_layer
):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    x = keras.layers.Flatten()(x)
    x = classifier_layer(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.cm as cm

last_conv_layer = model.layers[-4]
classifier_layer = model.layers[-1]

for idx, (images, output) in enumerate(test_generator):

    if idx == 1:
        break

    for i in range(batch_size):
        img = images[i] * 255

        image = np.expand_dims(images[i], axis=0)

        preds = model.predict(image)

        actual_value = np.argmax(output[i], axis=0)

        predicted_value = np.argmax(preds, axis=1)[0]

        confidence = preds[0][predicted_value] * 100

        # Generate class activation heatmap
        heatmap = make_gradcam_heatmap(
            image, model, last_conv_layer, classifier_layer
        )


        # We rescale heatmap to a range 0-255
        heatmap = np.uint8(255 * heatmap)

        # We use jet colormap to colorize heatmap
        jet = cm.get_cmap("jet")

        # We use RGB values of the colormap
        jet_colors = jet(np.arange(256))[:, :3]
        jet_heatmap = jet_colors[heatmap]

        # We create an image with RGB colorized heatmap
        jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
        jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
        jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

        # Superimpose the heatmap on original image
        superimposed_img = jet_heatmap * 0.6 + img
        superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

        plt.figure(figsize=(5,5))
        plt.imshow(superimposed_img)
        plt.show()

        print("Confidence", confidence, "%")
        print("Actual_value", idx_to_attribute[int(actual_value)])
        print("Predicted value", idx_to_attribute[int(predicted_value)])
    
        
    idx += 1

## STEEL DEFECT DATA

### Building model

In [ ]:
import keras
import tensorflow as tf
import keras.backend as K 

In [ ]:
def build_model(optimizer, learning_rate, out_type, categories):

    policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')#sets values to be float16 for nvidia 2000,3000 series GPUs, plus others im sure
    
    input_img = keras.layers.Input(shape=(224, 224, 3))
  
    x = keras.layers.Conv2D(8, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(input_img)
    x = keras.layers.Conv2D(8, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Conv2D(16, 2,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Conv2D(32, 2,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(32, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNormalization()(x)    

    x = keras.layers.Conv2D(16, 1, activation="selu",kernel_initializer="lecun_normal",dtype=policy)(x)
    x = keras.layers.Conv2D(1, 1, activation='linear')(x)
    x_map = keras.activations.sigmoid(x)

    y = keras.layers.GlobalAveragePooling2D()(x_map)
    y_out = keras.layers.Dense(categories, activation='softmax')(y)
    
    if optimizer == "SGD":
        #looks like pretty good but noisy results with no momentum, lets check 0.9...
        optimizer = keras.optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
    if optimizer == "RMSprop":
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    if optimizer == "Adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model = keras.models.Model(inputs=input_img, outputs=x_map)
    
    model.compile(loss="categorical_crossentropy",optimizer=optimizer, metrics=['accuracy','AUC'])

    return model


### Pre-processing

In [ ]:
import os
import tarfile
import shutil
import numpy as np

main_dataset_dir = '/content/drive/MyDrive/Major Project/Datasets/datasets'

model_dir = '/content/drive/MyDrive/Major Project/Models/V1'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [ ]:
filename = 'steel_defect_cls_products.tar.xz'
tar_file = os.path.join(dataset_dir, filename)

my_tar = tarfile.open(tar_file)
my_tar.extractall(dataset_dir) # specify which folder to extract to
my_tar.close()

In [ ]:
dataset = os.path.join(main_dataset_dir, 'steel_defect_cls_products')

new_dataset = os.path.join(main_dataset_dir, 'steel_defect_cls_products_cropped')

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

dataset_dir = dataset

destination_dir = new_dataset

if not os.path.exists(destination_dir):
	os.mkdir(destination_dir)

for data_split in os.listdir(dataset_dir):

	print('\n', data_split, '\n')

	datasplit_path = os.path.join(dataset_dir, data_split)

	destination_split = os.path.join(destination_dir, data_split)

	if not os.path.exists(destination_split):
		os.mkdir(destination_split)

	for category in os.listdir(datasplit_path):

		print('\n', category, '\n')

		category_path = os.path.join(datasplit_path, category)

		destination_category = os.path.join(destination_split, category)

		if not os.path.exists(destination_category):
			os.mkdir(destination_category)

		for image_name in tqdm(os.listdir(category_path)):

			image_path = os.path.join(category_path, image_name)

			image = cv2.imread(image_path)

			height, width, _ = image.shape

			new_width = int(width /6) 

			for idx in range(6):

				new_image = image[:, idx* new_width: (idx + 1) * new_width, : ]

				unique_values = len(np.unique(new_image))

				destination_image_path = os.path.join(destination_category, image_name.split('.')[0] + '_' + str(idx) + '.jpg')


				if unique_values > 120 and not os.path.exists(destination_image_path):

					cv2.imwrite(destination_image_path, new_image)

				

In [ ]:
generator = keras.preprocessing.image.ImageDataGenerator(
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            brightness_range = [0.75,1.0],
                            zoom_range=0.0,
                            rescale=1./255,
                            fill_mode="constant",
                            cval=0.0,
                            horizontal_flip=True,
                            vertical_flip=True,
                            validation_split=0.1,
                            dtype='float64',
                        )

In [ ]:
batch_size = 64

train_generator = generator.flow_from_directory( os.path.join(new_dataset, 'train'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='training') 
                                            
val_generator = generator.flow_from_directory( os.path.join(new_dataset, 'train'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='validation') 



In [ ]:
a = next(train_generator)
for img in a[0]:

    plt.imshow(img)
    plt.show()

In [ ]:
number_of_classes = len(train_generator.class_indices)

In [ ]:
map_model = build_model("SGD", 1e-3, "map", number_of_classes)
model = build_model("SGD", 1e-3, "classify", number_of_classes)
model.summary()

### Training the model

In [ ]:
epochs = 50

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=5, min_lr = 1e-8, factor=0.5, verbose=1)

history = model.fit(train_generator, 
                    epochs= epochs,
                    validation_data = val_generator,
                    verbose=1,
                    callbacks=[reduce_lr],
                    use_multiprocessing=True,
                    workers=-1)

In [ ]:
 import matplotlib.pyplot as plt

print("AUC")
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.show()

print("Loss")
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

print('Accuracy')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()


print('Learning rate')
plt.plot(history.history['lr'])
plt.show()

In [ ]:
model.save(os.path.join(model_dir, 'steel_V3(50eps).h5'))

### Testing the model

In [ ]:
for folder in os.listdir(os.path.join(new_dataset, 'train')):
    print(folder)
    print(len(os.listdir(os.path.join(new_dataset, 'train', folder))))

In [ ]:
test_generator = generator.flow_from_directory( os.path.join(new_dataset, 'test'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                shuffle=True) 

In [ ]:
attribute_to_idx = test_generator.class_indices
idx_to_attribute = {value:key for key,value in attribute_to_idx.items()}

print(attribute_to_idx)
print(idx_to_attribute)

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
scores_keys = ['loss', 'accuracy', 'auc']

for key,score in zip(scores_keys, scores):

    print(key, ':', score)

"""
epochs 50 - V1

loss : 0.8872455358505249
accuracy : 0.7154929637908936
auc : 0.8439496159553528
"""

"""
epochs 50 - V2

loss : 1.2701878547668457
accuracy : 0.34939759969711304
auc : 0.6427261233329773

"""

"""
v3 - epochs 50

loss : 0.9832906723022461
accuracy : 0.5500945448875427
auc : 0.8196025490760803
"""

In [ ]:
from google.colab.patches import cv2_imshow

for idx, (images, output) in enumerate(test_generator):

    if idx == 2:
        break

    for i in range(batch_size):

        image = images[i]

        cv2_imshow(image * 255)

        preds = model.predict(np.expand_dims(image, axis=0))

        actual_value = np.argmax(output[i], axis=0)

        predicted_value = np.argmax(preds, axis=1)[0]

        confidence = preds[0][predicted_value] * 100

        print("Confidence", confidence, "%")
        print("Actual_value", idx_to_attribute[int(actual_value)])
        print("Predicted value", idx_to_attribute[int(predicted_value)])
    
    idx += 1

#### Visualise Activation maps

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(
    img_array, model, last_conv_layer, classifier_layer
):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    x = keras.layers.Flatten()(x)
    x = classifier_layer(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.cm as cm

last_conv_layer = model.layers[-4]
classifier_layer = model.layers[-1]

for idx, (images, output) in enumerate(test_generator):

    if idx == 1:
        break

    for i in range(batch_size):
        img = images[i] * 255

        image = np.expand_dims(images[i], axis=0)

        preds = model.predict(image)

        actual_value = np.argmax(output[i], axis=0)

        predicted_value = np.argmax(preds, axis=1)[0]

        confidence = preds[0][predicted_value] * 100

        # Generate class activation heatmap
        heatmap = make_gradcam_heatmap(
            image, model, last_conv_layer, classifier_layer
        )


        # We rescale heatmap to a range 0-255
        heatmap = np.uint8(255 * heatmap)

        # We use jet colormap to colorize heatmap
        jet = cm.get_cmap("jet")

        # We use RGB values of the colormap
        jet_colors = jet(np.arange(256))[:, :3]
        jet_heatmap = jet_colors[heatmap]

        # We create an image with RGB colorized heatmap
        jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
        jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
        jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

        # Superimpose the heatmap on original image
        superimposed_img = jet_heatmap * 0.6 + img
        superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

        plt.figure(figsize=(5,5))
        plt.imshow(superimposed_img)
        plt.show()

        print("Confidence", confidence, "%")
        print("Actual_value", idx_to_attribute[int(actual_value)])
        print("Predicted value", idx_to_attribute[int(predicted_value)])
    
        
    idx += 1